In [ ]:
from semantic_kernel.contents import (
    FunctionCallContent,
    FunctionResultContent,
    StreamingTextContent,
)
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function
from openai import AsyncOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

True

In [125]:
endpoint = "https://models.inference.ai.azure.com/"
# endpoint = "https://models.github.ai/inference"
model = "gpt-4o-mini"  # "openai/gpt-4.1"
embeddings_model = "text-embedding-3-large"  # "text-embedding-3-small"
token = os.environ["OPENAI_API_KEY"]

### 🔧 Initialize OpenAI Client and Chat Completion Service

This cell sets up the connection to an OpenAI-compatible API endpoint and prepares the chat completion service used by the agent.

- `AsyncOpenAI`: Asynchronous client configured with a custom `base_url` (e.g., Azure or third-party provider) and an API key for authentication.
- `OpenAIChatCompletion`: A Semantic Kernel wrapper that enables chat-based completions using the specified model and client.

**Parameters:**
- `endpoint`: The base URL for the OpenAI-compatible API.
- `token`: API key for authentication.
- `model`: The identifier for the model to use (e.g., `gpt-4`, `gpt-4o`, or provider-specific).

This setup is required for enabling the `ChatCompletionAgent` to send and receive messages using the specified model.


In [126]:
client = AsyncOpenAI(base_url=endpoint, api_key=token)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(ai_model_id=model, async_client=client)

### 🧩 WebsiteChatbotPlugin

This class enables semantic search over a predefined list of web pages. It:

- Loads or builds an index of website content by scraping a list of important links.
- Chunks the text into smaller strings of 300.
- Uses OpenAI embeddings to represent text chunks.
- Provides a kernal function `search_website` method to retrieve relevant information based on user queries, 
  by computing the cosine similirties and fetching the most relevant answers.
- Finally, The model answers provided with information from the top answers.

In [ ]:
from website_scraper import WebScraper
import numpy as np
import pandas as pd


class WebsiteChatbotPlugin:
    """A simple plugin to retrieve relevant content from a website."""

    def __init__(self, client, embeddings_model):
        self.client = client
        self.embeddings_model = embeddings_model
        self.base_urls = [
            "https://mittspeciellabarn.se/",
            "https://mittspeciellabarn.se/community-support/",
            "https://mittspeciellabarn.se/activities/",
            "https://mittspeciellabarn.se/services/",
            "https://mittspeciellabarn.se/about-us/",
        ]
        self.df = None

    async def read_index(self, filename="website_index.csv"):
        try:
            self.df = pd.read_csv(filename)
            self.df["embeddings"] = self.df.embeddings.apply(eval).apply(np.array)
            print(f"✅ Loaded index from {filename}.")
        except FileNotFoundError:
            print(f"❌ File {filename} not found. Building index ...")
            chunks, links = self.build_index()
            embeddings = await self.get_embeddings(chunks)
            self.df = pd.DataFrame(
                {"url": links, "text": chunks, "embeddings": embeddings}
            )
            self.df.to_csv(filename, index=False)
            print(f"✅ Index built and saved to {filename}.")

    @kernel_function(
        description="Provides relevant information and resources from the website."
    )
    async def search_website(self, user_query, top_k=3):
        try:
            print("🔍 Searching for relevant information...")
            # Extract actual query text from the dict
            if isinstance(user_query, dict):
                user_query = list(user_query.values())[0]
            user_query_embedding = await self.get_embeddings([user_query])
            similarities = self.df.embeddings.apply(
                lambda e: self.cosine_similarity(e, user_query_embedding[0])
            )
            top_indices = similarities.nlargest(top_k).index
            top_texts = self.df.loc[top_indices, "text"].tolist()
            print("✅ Found relevant information.")
            return top_texts
        except Exception as e:
            print("❌ Error in search_website function:", e)
            print("user_query:", user_query)
            return ["could not find any relevant information."]

    ################################################
    ############### helper functions ###############
    ################################################

    def build_index(self, max_level=4):
        """Build an index of the website content."""

        unique_links_dict = WebScraper().scrape(self.base_urls, max_level=max_level)
        chunks = []
        links = []
        for url, text in unique_links_dict.items():
            if text.strip():
                chunked = self.chunk_text(text)
                chunks.extend(chunked)
                links.extend([url] * len(chunked))
        print(
            f"✅ Created {len(chunks)} text chunks ready for embedding. Compared to the original {len(unique_links_dict.keys())} text chunks."
        )
        return chunks, links

    def chunk_text(self, text, max_tokens=300):
        """Chunk the text into smaller pieces of tokens."""
        words = text.split()
        chunks = []
        for i in range(0, len(words), max_tokens):
            chunk = " ".join(words[i : i + max_tokens])
            if chunk.strip():
                chunks.append(chunk)
        return chunks

    async def get_embeddings(self, texts):
        embeddings = await self.client.embeddings.create(input=texts, model=self.embeddings_model, dimensions=1536)
        embeddings = [e.embedding for e in embeddings.data]
        return embeddings

    async def get_embedding(self, text):
        embeddings = await self.client.embeddings.create(
            input=[text], model=self.embeddings_model, dimensions=1536)
        return embeddings.data[0].embedding

    def cosine_similarity(self, a, b):
        from scipy.spatial.distance import cosine

        # Note that scipy.spatial.distance.cosine computes the cosine distance, which is 1 - cosine similarity.
        # https://en.wikipedia.org/wiki/Cosine_similarity#Cosine_distance
        return 1 - cosine(a, b)

### 🤖 ChatCompletionAgent Setup

This section initializes the AI agent that will assist users in querying the *Mitt Speciella Barn* website.

- **AGENT_INSTRUCTIONS**: Provides role guidance and interaction style for the assistant.
- **WebsiteChatbotPlugin**: Loads the website index to enable semantic search.
- **ChatCompletionAgent**: Wraps the chat completion service with the plugin and instructions to act as a helpful assistant focused on website content related to children with disabilities.


In [128]:
AGENT_INSTRUCTIONS = """

You are a helpful AI Agent that can assist users in finding relevant information and resources regarding children with special needs from the "MittSpeciellaBarn" website.

Important: When users specify a topic or query, always focus on providing information related to that topic from the website. 
Only suggest general resources when an answer is not relevant from the website. prioritize user preferences. 

When the conversation begins, introduce yourself with this message:
"Hello! I'm your Mitt Speciella Barn assistant. I can help you find information resources and activites related to support for children and individuals with disabilities. Here are some things you can ask me:
1. How to apply for financial support for children with disabilities.
2. Information about specific services or activities.
3. Details about community support and resources.

What can I help you with today?"

Always  
Answer in the same langague as the question and provide a link (if exists) to the relevant page on the website at the end of the response.
If they mention a specific topic like "financial support" or "community activities," focus your assistance on that topic rather than suggesting unrelated resources. 
"""

wcp = WebsiteChatbotPlugin(client=client, embeddings_model=embeddings_model)
await wcp.read_index(filename="website_index.csv")

agent = ChatCompletionAgent(
    service=chat_completion_service,
    plugins=[wcp],
    name="WebsiteAgent",
    instructions=AGENT_INSTRUCTIONS,
)

✅ Loaded index from website_index.csv.


In [129]:
# A list of example queries covering different user intents and languages.

user_inputs = [
    "Hur kan jag ansöka om ekonomiskt stöd för mitt barn med funktionsnedsättning?",
    "اين الخدمات المتاحة لدعم الأطفال ذوي الاحتياجات الخاصة؟",
    "what are the activities that I can do for my child with special needs in Helsingborg?",
]

### 🧪 Testing the Website Agent with Multilingual Inputs

This section runs a simulated interaction with the `ChatCompletionAgent` using various user queries in Swedish, Arabic, and English.

- **main()**:
  - Iterates over each input and streams the assistant's response.
  - Tracks and reconstructs streamed function calls and their results.
  - Displays the user query, assistant response, and any function call details in a styled HTML format for easy readability.
- **Thread Management**: Maintains conversation continuity across multiple queries by reusing the `thread` object.


In [130]:
from IPython.display import display, HTML
import json
import html

# Emoji icons for accessibility and readability
ICONS = {
    "user": "👤",
    "agent": "🤖",
    "function": "🛠️",
    "result": "📄"
}

async def main():
    thread: ChatHistoryAgentThread | None = None

    for user_input in user_inputs:
        escaped_input = html.escape(user_input)
        html_output = f"""
        <section style="margin:1em 0; font-family:Arial, sans-serif;">
            <div style="font-weight:bold;">{ICONS['user']} User:</div>
            <div style="margin-left:1em;">{escaped_input}</div>
        </section>
        """

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(messages=user_input, thread=thread):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments

                elif isinstance(item, FunctionResultContent):
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args, indent=2)
                        except Exception:
                            pass
                        function_calls.append(
                            f"<div><strong>{ICONS['function']} Calling:</strong> {current_function_name}"
                            f"<pre style='background:#eee; padding:0.5em; border-radius:5px;'>{html.escape(formatted_args)}</pre></div>"
                        )
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(
                        f"<div><strong>{ICONS['result']} Result:</strong><pre style='background:#f0f0f0; padding:0.5em; border-radius:5px;'>{html.escape(str(item.result))}</pre></div>"
                    )

                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        # Collapsible function call section
        if function_calls:
            html_output += f"""
            <details style="margin:1em 0;">
                <summary style="cursor:pointer; font-weight:bold; color:#336699;">
                    {ICONS['function']} Function Calls (click to expand)
                </summary>
                <div style="margin-top:0.5em; font-size:0.95em;">
                    {"<hr>".join(function_calls)}
                </div>
            </details>
            """

        # Assistant response
        html_output += f"""
        <section style="margin:1em 0;">
            <div style="font-weight:bold;">{ICONS['agent']} {html.escape(agent_name or 'Assistant')}:</div>
            <div style="margin-left:1em; white-space:pre-wrap;">{html.escape(''.join(full_response))}</div>
        </section>
        <hr style="border:none; border-top:1px solid #ccc; margin:2em 0;">
        """

        display(HTML(html_output))

await main()

🔍 Searching for relevant information...


🔍 Searching for relevant information...


🔍 Searching for relevant information...
